In [1]:
import sys
import numpy as np
from functools import reduce

from pyscf.lib import logger
from pyscf import lib

from ad_afqmc.lno.cc import LNOCCSD

from pyscf import gto, scf, mp, cc, fci

log = logger.Logger(sys.stdout, 6)

a = 0.9
nH = 6
atoms = ""
for i in range(nH):
    atoms += f"H {i*a} 0 0 \n"

mol = gto.M(atom=atoms, basis="sto6g", verbose=4)
mf = scf.RHF(mol).density_fit()
mf.kernel()

frozen = 0
mmp = mp.MP2(mf,frozen=frozen)
mmp.kernel()[0]

# cc
mycc = cc.CCSD(mf)
mycc.kernel()
et = mycc.ccsd_t()

# fci
cisolver = fci.FCI(mf)
fci_ene, fci_vec = cisolver.kernel()

print(f'rhf energy is {mf.e_tot}')
print(f"ccsd energy is {mycc.e_tot}")
print(f"ccsd_t energy is {mycc.e_tot+et}")
print(f"fci_ene: {fci_ene}", flush=True)

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#1882-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Tue Apr 22 15:04:27 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 6
[INPUT] num. electrons = 6
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00

In [4]:
from ad_afqmc import pyscf_interface

mo_file="mo.npz"
chol_file="chol"
amp_file="amp.npz"
pyscf_interface.prep_afqmc(mf,mo_file=mo_file,amp_file=amp_file,chol_file=chol_file)

#
# Preparing AFQMC calculation
# Calculating Cholesky integrals
# Decomposing ERI with DF
# Decomposing ERI with DF
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (3, 3)
# Number of basis functions: 6
# Number of Cholesky vectors: 54
#


In [6]:
from ad_afqmc.corr_sample_test import corr_sample
from ad_afqmc import mpi_jax, config
import time

options = {
    "dt": 0.01,
    "n_exp_terms": 6,
    "n_walkers": 50,
    "n_runs": 10,
    "rlx_steps": 0,
    "prop_steps": 10,
    "seed": 2,
    "walker_type": "rhf",
    "trial": "rhf",
    "use_gpu": False,
    "free_proj": False,
}

nwalkers = options["n_walkers"]
dt = options["dt"]
seed = options["seed"]
rlx_steps = options["rlx_steps"]
prop_steps = options["prop_steps"]
n_runs = options["n_runs"]

mo_file="mo.npz"
chol_file="chol"
amp_file="amp.npz"

config.setup_jax()
MPI = config.setup_comm()

init_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()  # Process rank
size = comm.Get_size()  # Total number of processes

ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = mpi_jax._prep_afqmc(options,mo_file=mo_file,amp_file=amp_file,chol_file=chol_file)

prop_data_init, ham_data_init = \
    corr_sample.init_prop(ham_data, ham, prop, trial, wave_data, seed, MPI)


if options["trial"] == "rhf":
    print(f'# relaxation from mean-field object using {nwalkers*size} walkers')
if options["trial"] == "cisd":
    print(f'# relaxation from ccsd object using {nwalkers*size} walkers')
print('# rlx_step \t energy ')
print(f'    {0}  \t \t {prop_data_init["e_estimate"]:.6f}')

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 6
# nelec: (3, 3)
#
# dt: 0.01
# n_exp_terms: 6
# n_walkers: 50
# n_runs: 10
# rlx_steps: 0
# prop_steps: 10
# seed: 2
# walker_type: rhf
# trial: rhf
# use_gpu: False
# free_proj: False
# n_prop_steps: 50
# n_ene_blocks: 50
# n_sr_blocks: 1
# n_blocks: 50
# n_eql: 1
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# free_projection: False
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#
# initial energy: -3.186365
# relaxation from mean-field object using 50 walkers
# rlx_step 	 energy 
    0  	 	 -3.186365


In [21]:
walker_test = trial.get_init_walkers(wave_data, 2, restricted=True)

In [28]:
print(wave_data["mo_coeff"])

[[-1.00000000e+00  2.12266284e-16 -4.64163810e-16]
 [-2.12266284e-16 -1.00000000e+00 -7.62997678e-17]
 [ 4.64163810e-16  7.62997678e-17 -1.00000000e+00]
 [ 3.96173559e-16  2.97978703e-16  1.36538564e-16]
 [ 6.32865877e-18 -7.89682975e-17 -1.80215842e-17]
 [-1.68789461e-16 -4.39110441e-17  2.10108405e-16]]


In [23]:
from jax import numpy as jnp
olp = jnp.linalg.det(wave_data["mo_coeff"].T.conj() @ walker_test) ** 2
print(olp)

[1.+0.j 1.+0.j]


In [29]:
walker_test[0]

Array([[ 1.07199371e-31+0.j,  1.00000000e+00+0.j,  1.99814555e-47+0.j],
       [-2.28959093e-16+0.j, -9.56296963e-32+0.j, -1.00000000e+00+0.j],
       [ 1.00000000e+00+0.j, -9.86076132e-32+0.j, -2.28959093e-16+0.j],
       [-4.81751629e-17+0.j, -3.96173559e-16+0.j,  1.28811689e-16+0.j],
       [ 1.94331418e-17+0.j, -6.32865877e-18+0.j, -5.94803572e-16+0.j],
       [-2.47755569e-16+0.j,  1.68789461e-16+0.j,  8.70227898e-17+0.j]],      dtype=complex128)

In [20]:
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)

KeyError: 'rot_h1'

In [10]:
trial.calc_overlap(prop_data_init["walkers"], wave_data)

Array([1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j], dtype=complex128)

In [18]:
print(mf.mo_coeff)

[[ 0.1511032  -0.33821699  0.47420513 -0.65207559 -0.71465345 -0.47791398]
 [ 0.26308475 -0.41474068  0.27818239  0.3566349   0.95178797  0.9508896 ]
 [ 0.36008369 -0.13884193 -0.44609829  0.56939293 -0.41244245 -1.23234583]
 [ 0.36008369  0.13884193 -0.44609829 -0.56939293 -0.41244245  1.23234583]
 [ 0.26308475  0.41474068  0.27818239 -0.3566349   0.95178797 -0.9508896 ]
 [ 0.1511032   0.33821699  0.47420513  0.65207559 -0.71465345  0.47791398]]


In [15]:
print(prop_data_init["walkers"][0])

[[ 1.07199371e-31+0.j  1.00000000e+00+0.j  1.99814555e-47+0.j]
 [-2.28959093e-16+0.j -9.56296963e-32+0.j -1.00000000e+00+0.j]
 [ 1.00000000e+00+0.j -9.86076132e-32+0.j -2.28959093e-16+0.j]
 [-4.81751629e-17+0.j -3.96173559e-16+0.j  1.28811689e-16+0.j]
 [ 1.94331418e-17+0.j -6.32865877e-18+0.j -5.94803572e-16+0.j]
 [-2.47755569e-16+0.j  1.68789461e-16+0.j  8.70227898e-17+0.j]]
